In [ ]:
import os
import urllib.parse
import dotenv
import networkx as nx
from typing import List

# Pydantic周りのバージョン競合を避けるため、直接importします
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# 可視化用のライブラリ
from pyvis.network import Network
from IPython.display import display, IFrame

# 1. 環境設定 (.envファイルから読み込み)
dotenv.load_dotenv()
# もし.envを使わない場合は、以下のように直接入れてください
# os.environ["OPENAI_API_KEY"] = "sk-..." 

# ---------------------------------------------------------
# 2. 「何を抜き出すか」を決める (スキーマ定義)
# ---------------------------------------------------------
# ここで定義した型に合わせてLLMがデータを探してくれます
class Entity(BaseModel):
    name: str = Field(..., description="エンティティの名前（人物、組織、技術、場所など）")
    type: str = Field(..., description="エンティティの種類")

class Relation(BaseModel):
    source: str = Field(..., description="関係の主体（始点ノード名）")
    target: str = Field(..., description="関係の客体（終点ノード名）")
    relation_type: str = Field(..., description="関係性の説明（例: works_at, located_in, uses）")

class KnowledgeGraph(BaseModel):
    entities: List[Entity]
    relations: List[Relation]

# ---------------------------------------------------------
# 3. LLMに働いてもらう (Extraction)
# ---------------------------------------------------------
# サンプルテキスト（今回は私の自己紹介文を使ってみます）
sample_text = """
宮脇彰梧はルミナイ株式会社（Lluminai）のエンジニアであり、現在は大学院でマルチモーダルAIの研究を行っている。
ルミナイ株式会社は東京に拠点を置き、生成AI技術を活用したソリューション開発を行っている。
宮脇の研究テーマにはLLMやVision-Languageモデルが含まれる。
GraphRAGは、マイクロソフトによって提案された新しいRAGのアプローチである。
"""

# モデル準備（安くて速いgpt-4o-miniがおすすめ）
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
structured_llm = llm.with_structured_output(KnowledgeGraph)

# プロンプト「あなたはデータサイエンティストです。グラフを作ってください」
system_prompt = """
あなたはデータサイエンティストです。
与えられたテキストから、知識グラフを構築するためのエンティティとリレーションを抽出してください。
エンティティは一貫性のある名前を使用してください。
"""
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
])

# 実行！
print("🚀 テキストから知識を抽出中...")
chain = prompt | structured_llm
graph_data = chain.invoke({"input": sample_text})
print(f"✅ 抽出完了: {len(graph_data.entities)} 個のエンティティ, {len(graph_data.relations)} 本の関係性")

# ---------------------------------------------------------
# 4. グラフを構築する (Graph Construction)
# ---------------------------------------------------------
G = nx.DiGraph()

# ノード（点）を追加
for entity in graph_data.entities:
    G.add_node(
        entity.name, 
        label=entity.name, 
        title=f"Type: {entity.type}",
        group=entity.type # タイプごとに色を変える
    )

# エッジ（線）を追加
for rel in graph_data.relations:
    G.add_edge(
        rel.source, 
        rel.target, 
        title=rel.relation_type, 
        label=rel.relation_type,
        arrows="to"
    )

# Pyvis設定
# 【重要】notebook=Falseにして、HTML生成に専念させます
net = Network(
    notebook=False, 
    height="600px", 
    width="100%", 
    bgcolor="#222222", # ダークモードでカッコよく
    font_color="white", 
    cdn_resources="in_line",
    directed=True
)

net.from_nx(G)
net.force_atlas_2based() # 物理演算ON！

# HTMLファイルとして保存
html_file = "knowledge_graph.html"
net.save_graph(html_file)
print(f"📁 グラフを保存しました: {html_file}")

# ---------------------------------------------------------
# 5. いざ表示！ (Visualization Fix)
# ---------------------------------------------------------
# ここがポイント。VS Codeなどはローカルファイルの表示をブロックしがち。
# なので、HTMLの中身を「文字列」として読み込んで、ブラウザに直接流し込みます。

with open(html_file, "r", encoding="utf-8") as f:
    html_string = f.read()

# 文字列をURI形式にエンコード
html_encoded = urllib.parse.quote(html_string)

print("✨ グラフを表示します（グリグリ動かしてみて！）")
display(IFrame(src=f"data:text/html;charset=utf-8,{html_encoded}", width="100%", height="650px"))